In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [2]:
loader = PyPDFLoader('./data/2024_KB_부동산_보고서_최종.pdf')
documents = loader.load()
text_splitter = SemanticChunker(embeddings=OpenAIEmbeddings())
chunks = text_splitter.split_documents(documents)
print(len(chunks))

164


In [3]:
persist_dir = './data/rag_practice_chroma'
vectordb = Chroma.from_documents(documents=chunks, embedding=OpenAIEmbeddings(), persist_directory=persist_dir)
print(vectordb._collection.count())

164


In [4]:
retriever = vectordb.as_retriever(search_kwargs={'k':3})

In [7]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [5]:
template = '''당신은 KB 부동산 보고서 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.
컨텍스트: {context}'''

prompt = ChatPromptTemplate.from_messages(
    [('system', template),
     ('placeholder', '{chat_history}'),
     ('human', '{question}')]
)

model = ChatOpenAI(model='gpt-5-nano', temperature=0)

In [6]:
print(prompt.format(context='컨텍스트 예시', chat_history=['대화 기록 예시1', '대화 기록 예시2'], question='질문 예시'))

System: 당신은 KB 부동산 보고서 전문가입니다. 다음 정보를 바탕으로 사용자의 질문에 답변해주세요.
컨텍스트: 컨텍스트 예시
Human: 대화 기록 예시1
Human: 대화 기록 예시2
Human: 질문 예시


In [8]:
chain = (
    RunnablePassthrough.assign(context=lambda x: format_docs(retriever.invoke(x['question'])))
    | prompt
    | model
    | StrOutputParser()
)

In [9]:
chat_history = ChatMessageHistory()

chain_with_memory = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key='question',
    history_messages_key='chat_history'
)

In [10]:
def chat_with_bot():
    session_id = 'user_session'
    print('KB 부동산 보고서 챗봇입니다. 질문해주세요. (종료하려면 "종료"를 입력해주세요.)')
    while True:
        user_input = input('사용자: ')
        if user_input == '종료' or user_input.lower() == 'quit':
            break

        response = chain_with_memory.invoke(
            {'question':user_input},
            {'configurable':{'session_id':session_id}}
        )

        print(f'챗봇: {response}')

In [11]:
chat_with_bot()

KB 부동산 보고서 챗봇입니다. 질문해주세요. (종료하려면 "종료"를 입력해주세요.)
챗봇: 멋진 실습이네요. 랭체인으로 부동산 KB를 다루는 챗봇을 만들 때의 기본 흐름과 바로 따라할 수 있는 가이드라인을 정리해 드릴게요. 제공하신 자료(예: 그림Ⅲ-17 GTX-A 노선 조기 개통 추진 현황, 3기 신도시 등)처럼 도메인 문서를 다루는 QA 봇으로 구성하는 것을 목표로 하면 좋습니다.

1) 기본 설계 방향
- 목적 정의: 경기 남부권 반도체 클러스터/GTX-A 노선 관련 실무 질의에 대해 신뢰 가능한 출처를 근거로 답변하고, 필요 시 출처를 함께 제시.
- 데이터 범위: 제공된 보고서·자료(PDF/Text) 및 요약 노트 등을 문서로 수집하고 카테고리(Tag)별로 정리.
- 응답 형식: 간단 요약 vs 상세 설명 중 선택 가능. 항상 출처를 표시하도록 설계.
- 다중-turn 대화: “출처 확인 → 요약 → 추가 정보 질의” 같은 흐름으로 대화 유지.

2) 데이터 준비 및 인덱싱
- 문서 수집: 그림 제목, 그림 번호(예: 그림Ⅲ-17), 부제목, 핵심 수치 등을 태깅.
- 텍스트 추출/정제: PDF에서 텍스트 추출 시 표/도표는 텍스트 요약으로 대체하되 원문 출처를 남김.
- 메타데이터 추가: 출처 문서명, 섹션, 그림 번호, 날짜, 담당 부서 등.
- 벡터 상에 인덱싱: 임베딩을 만들어 벡터 스토어에 저장.

3) 기술 스택 구성(간단한 예)
- 텍스트 인덱싱 및 검색: LangChain + 벡터 스토어(예: FAISS)
- 임베딩 모델: 한국어에 잘 맞는 모델 또는 OpenAI의 텍스트 임베딩
- LLM: 한국어를 잘 이해하는 모델(OpenAI의 gpt-4/gpt-3.5-turbo 등 또는 로컬 모델)
- 대화 체인: RetrievalQA(혹은 chat 기반 대화 체인)로 질의-응답
- 예시 구성 흐름
  - 문서 로더 → 임베딩 → 벡터 스토어 생성
  - RetrievalQA 또는 Chat 기반 체인에 벡터 리트리버 연결
  - 사용자가 질의하면